# Merging the Child Survey

In [32]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [1]:
pip install pyreadstat 

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)
Note: you may need to restart the kernel to use updated packages.


In [2]:
#the neccesary library 
import pyreadstat

import pandas as pd 

In [3]:
#opening the file Child file 
year18, meta = pyreadstat.read_sav('active_lives_survey_child_201819_main_shared.sav')
year17, meta = pyreadstat.read_sav('active_lives_survey_child_201718_main_shared.sav')
year19, meta = pyreadstat.read_sav('active_lives_survey_child_2019-20_main_shared.sav')
year20, meta = pyreadstat.read_sav('active_lives_survey_child_2020-21_main_shared.sav')

In [4]:
meta_dict = dict(zip(meta.column_names, meta.column_labels))


In [5]:
year17.head()

,Respondent_Serial,overall,xsurvey_year,term,wt_gross,xstrata,xcluster,xSCHYR_5_2,SCHYR_11,SCHYR_5_1,...,Progress8_GR,Absence_Overall_GR4,Absence_Persistent_GR6,Exclusions_Permanent_GR4,Exclusions_Fixed_GR5,Daily_mile,LDP_combined,LDP_proxy,STP_NC,SCHYR_4
0,14.0,1.0,1.0,1.0,100.987513,11017.0,300213.0,4.0,8.0,4.0,...,5.0,4.0,5.0,3.0,5.0,NaN,0.0,0.0,27.0,3.0
1,15.0,1.0,1.0,1.0,308.262470,12581.0,317310.0,2.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,8.0,28.0,2.0
2,16.0,1.0,1.0,1.0,767.461680,13179.0,302104.0,1.0,2.0,1.0,...,NaN,3.0,3.0,1.0,1.0,2.0,0.0,0.0,27.0,1.0
3,167.0,1.0,1.0,1.0,53.243303,11500.0,307202.0,3.0,6.0,3.0,...,NaN,2.0,2.0,1.0,1.0,2.0,0.0,0.0,14.0,2.0
4,168.0,1.0,1.0,1.0,53.243303,11500.0,307202.0,3.0,6.0,3.0,...,NaN,2.0,2.0,1.0,1.0,2.0,0.0,0.0,14.0,2.0


In [6]:
keep_variables = ['Respondent_Serial','xsurvey_year','SCHYR_11','eth6','DISAB2','age_11',
                   'STP_NC','Schoolestab','Urbanrural2','mins_modplus_everywhere_week_BADMINTON_CC023',
                   'onceawk_modplus_everywhere_GR_BADMINTON_CC023','PL_Easy_bc_ans','PL_GdMe_bc_ans','Trust_bc','SchM_c','gend3']



In [7]:
# Find column names containing "adult"
matching_columns = year20.columns[year20.columns.str.contains('IMD', case=False)]

# Print the matching column names
print(matching_columns.tolist())

['IMDDECILE', 'IMD10_GR3', 'IMD_QUINTILE']


The long-format merging method is useful when the data for each year is different, which is often the case in sports participation data. For example, participants may participate in different sports or at different levels of competition each year, or their levels of participation may vary. By organizing the data in this way, it is possible to examine changes in participation patterns over time for each participant, as well as to analyze overall trends in sports participation for the sample.

In [8]:
# List of variables to keep
keep_variables17 = ['Respondent_Serial', 'xsurvey_year', 'SCHYR_11', 'eth6', 'DISAB2', 'age_11',
                  'STP_NC', 'Schoolestab', 'Urbanrural2', 'mins_modplus_everywhere_week_BADMINTON_CC023',
                  'onceawk_modplus_everywhere_GR_BADMINTON_CC023','PL_Easy_bc_ans','PL_GdMe_bc_ans','Trust_bc','SchM_c','gend3','IMDDECILE']

# Keeping variables in each dataframe
year17 = year17[keep_variables17]
year18 = year18[keep_variables17]

In [9]:
keep_variables = ['Respondent_Serial', 'xsurvey_year', 'SCHYR_11', 'eth6', 'Disab_All_POP', 'age_11',
                  'STP_NC', 'Schoolestab', 'urbanrural2', 'mins_modplus_everywhere_week_BADMINTON_CC023',
                  'onceawk_modplus_everywhere_GR_BADMINTON_CC023','PL_Easy_bc_ans','PL_GdMe_bc_ans','Trust_bc','SchM_c','gend3','IMDDECILE']



year19 = year19[keep_variables]
year20 = year20[keep_variables]

In [10]:
var_names = ['serial','survey_year','school_year','ethnicity','disability',
             'age','district','school_type','urban_rural','mins_badminton','bamdinton_onceweek','Sports_easy','Understand_sports_important','Trust_peers','School_meals']

In [11]:
# Define the variable names
var_names = ['serial', 'survey_year', 'school_year', 'ethnicity', 'disability',
             'age', 'district', 'school_type', 'urban_rural', 'mins_badminton', 'badminton_onceweek',
             'Sports_easy', 'Understand_sports_important', 'Trust_peers', 'School_meals','Gender','IMD_child']

# Replace 'year17', 'year18', 'year19', 'year20' with your actual dataframes
year17.rename(columns=dict(zip(year17.columns, var_names)), inplace=True)
year18.rename(columns=dict(zip(year18.columns, var_names)), inplace=True)
year19.rename(columns=dict(zip(year19.columns, var_names)), inplace=True)
year20.rename(columns=dict(zip(year20.columns, var_names)), inplace=True)

In [12]:
year20

,serial,survey_year,school_year,ethnicity,disability,age,district,school_type,urban_rural,mins_badminton,badminton_onceweek,Sports_easy,Understand_sports_important,Trust_peers,School_meals,Gender,IMD_child
0,212.0,4.0,8.0,NaN,NaN,NaN,21.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0
1,7167.0,4.0,7.0,3.0,1.0,16.0,21.0,1.0,1.0,NaN,2.0,1.0,NaN,4.0,2.0,2.0,6.0
2,7571.0,4.0,9.0,1.0,NaN,13.0,21.0,1.0,1.0,NaN,1.0,2.0,1.0,1.0,NaN,2.0,6.0
3,7590.0,4.0,9.0,1.0,1.0,13.0,21.0,1.0,1.0,NaN,2.0,2.0,1.0,1.0,1.0,2.0,6.0
4,13838.0,4.0,8.0,1.0,NaN,13.0,25.0,2.0,2.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86823,148562.0,4.0,5.0,1.0,2.0,10.0,19.0,1.0,2.0,0.0,2.0,2.0,3.0,2.0,NaN,1.0,6.0
86824,191843.0,4.0,7.0,1.0,2.0,12.0,32.0,2.0,1.0,310.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
86825,140611.0,4.0,7.0,6.0,2.0,12.0,42.0,2.0,1.0,0.0,2.0,3.0,2.0,2.0,1.0,1.0,3.0
86826,164065.0,4.0,8.0,1.0,1.0,13.0,10.0,2.0,1.0,0.0,2.0,1.0,1.0,4.0,1.0,NaN,7.0


In [13]:
year17['year'] = 1
year18['year'] = 2
year19['year'] = 3
year20['year'] = 4


var_names = ['serial', 'survey_year', 'school_year', 'ethnicity', 'disability',
             'age', 'district', 'school_type', 'urban_rural', 'mins_badminton', 'badminton_onceweek']

year17.rename(columns={'Respondent_Serial': var_names[0], 'xsurvey_year': var_names[1],
                       'SCHYR_11': var_names[2], 'eth6': var_names[3],
                       'DISAB2': var_names[4], 'age_11': var_names[5],
                       'STP_NC': var_names[6], 'Schoolestab': var_names[7],
                       'Urbanrural2': var_names[8], 'mins_modplus_everywhere_week_BADMINTON_CC023': var_names[9],
                       'onceawk_modplus_everywhere_GR_BADMINTON_CC023': var_names[10]}, inplace=True)

year18.rename(columns={'Respondent_Serial': var_names[0], 'xsurvey_year': var_names[1],
                       'SCHYR_11': var_names[2], 'eth6': var_names[3],
                       'DISAB2': var_names[4], 'age_11': var_names[5],
                       'STP_NC': var_names[6], 'Schoolestab': var_names[7],
                       'Urbanrural2': var_names[8], 'mins_modplus_everywhere_week_BADMINTON_CC023': var_names[9],
                       'onceawk_modplus_everywhere_GR_BADMINTON_CC023': var_names[10]}, inplace=True)

year19.rename(columns={'Respondent_Serial': var_names[0], 'xsurvey_year': var_names[1],
                       'SCHYR_11': var_names[2], 'eth6': var_names[3],
                       'Disab_All_POP': var_names[4], 'age_11': var_names[5],
                       'STP_NC': var_names[6], 'Schoolestab': var_names[7],
                       'urbanrural2': var_names[8], 'mins_modplus_everywhere_week_BADMINTON_CC023': var_names[9],
                       'onceawk_modplus_everywhere_GR_BADMINTON_CC023': var_names[10]}, inplace=True)

year20.rename(columns={'Respondent_Serial': var_names[0], 'xsurvey_year': var_names[1],
                       'SCHYR_11': var_names[2], 'eth6': var_names[3],
                       'Disab_All_POP': var_names[4], 'age_11': var_names[5],
                       'STP_NC': var_names[6], 'Schoolestab': var_names[7],
                       'urbanrural2': var_names[8], 'mins_modplus_everywhere_week_BADMINTON_CC023': var_names[9],
                       'onceawk_modplus_everywhere_GR_BADMINTON_CC023': var_names[10]}, inplace=True)

year17['year'] = 1
year18['year'] = 2
year19['year'] = 3
year20['year'] = 4


In [14]:
year20

,serial,survey_year,school_year,ethnicity,disability,age,district,school_type,urban_rural,mins_badminton,badminton_onceweek,Sports_easy,Understand_sports_important,Trust_peers,School_meals,Gender,IMD_child,year
0,212.0,4.0,8.0,NaN,NaN,NaN,21.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,4
1,7167.0,4.0,7.0,3.0,1.0,16.0,21.0,1.0,1.0,NaN,2.0,1.0,NaN,4.0,2.0,2.0,6.0,4
2,7571.0,4.0,9.0,1.0,NaN,13.0,21.0,1.0,1.0,NaN,1.0,2.0,1.0,1.0,NaN,2.0,6.0,4
3,7590.0,4.0,9.0,1.0,1.0,13.0,21.0,1.0,1.0,NaN,2.0,2.0,1.0,1.0,1.0,2.0,6.0,4
4,13838.0,4.0,8.0,1.0,NaN,13.0,25.0,2.0,2.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,10.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86823,148562.0,4.0,5.0,1.0,2.0,10.0,19.0,1.0,2.0,0.0,2.0,2.0,3.0,2.0,NaN,1.0,6.0,4
86824,191843.0,4.0,7.0,1.0,2.0,12.0,32.0,2.0,1.0,310.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,4
86825,140611.0,4.0,7.0,6.0,2.0,12.0,42.0,2.0,1.0,0.0,2.0,3.0,2.0,2.0,1.0,1.0,3.0,4
86826,164065.0,4.0,8.0,1.0,1.0,13.0,10.0,2.0,1.0,0.0,2.0,1.0,1.0,4.0,1.0,NaN,7.0,4


In [15]:

# concatenate the data frames with a hierarchical index
child = pd.concat([year17, year18,year19, year20], keys=['1', '2', '3','4'])

# reset the index
child = child.reset_index(level=0, drop=True)


In [16]:
child

,serial,survey_year,school_year,ethnicity,disability,age,district,school_type,urban_rural,mins_badminton,badminton_onceweek,Sports_easy,Understand_sports_important,Trust_peers,School_meals,Gender,IMD_child,year
0,14.0,1.0,8.0,3.0,0.0,14.0,27.0,1.0,1.0,0.0,2.0,1.0,4.0,3.0,NaN,3.0,2.0,1
1,15.0,1.0,4.0,5.0,1.0,11.0,28.0,3.0,1.0,0.0,2.0,3.0,4.0,4.0,NaN,NaN,3.0,1
2,16.0,1.0,2.0,1.0,0.0,7.0,27.0,1.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,1
3,167.0,1.0,6.0,1.0,0.0,10.0,14.0,1.0,2.0,0.0,2.0,2.0,1.0,1.0,NaN,2.0,6.0,1
4,168.0,1.0,6.0,1.0,1.0,10.0,14.0,1.0,2.0,0.0,2.0,3.0,NaN,3.0,NaN,2.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86823,148562.0,4.0,5.0,1.0,2.0,10.0,19.0,1.0,2.0,0.0,2.0,2.0,3.0,2.0,NaN,1.0,6.0,4
86824,191843.0,4.0,7.0,1.0,2.0,12.0,32.0,2.0,1.0,310.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,4
86825,140611.0,4.0,7.0,6.0,2.0,12.0,42.0,2.0,1.0,0.0,2.0,3.0,2.0,2.0,1.0,1.0,3.0,4
86826,164065.0,4.0,8.0,1.0,1.0,13.0,10.0,2.0,1.0,0.0,2.0,1.0,1.0,4.0,1.0,NaN,7.0,4


In [17]:
# save the merged data to a new file
child.to_csv('child.csv', index=False)


# Cleaning the Child Survey

In [20]:
unique_values = child['serial'].nunique()
print("Number of unique values in 'Seriel' column:", unique_values)


#around 100k respondents who repeated the survey, can't simply do pooled blindly

Number of unique values in 'Seriel' column: 212021


In [21]:
# Change values from 2 to 0 in the 'badminton_onceweek' column of the 'child' DataFrame
child.loc[child['badminton_onceweek'] == 2, 'badminton_onceweek'] = 0

In [22]:
#function to set anything less than 0 to NAN as stated in the survey data 
def set_negative_to_nan(dataframe):
    dataframe[dataframe < 0] = np.nan
    return dataframe

# Apply the function to set negative values to NaN for all columns
full = set_negative_to_nan(child)

In [23]:
# Remove the 'disability' column
full = full.drop('disability', axis=1)

In [24]:
# Separate continuous and categorical columns
continuous_cols = ['mins_badminton', 'year', 'school_year','age']
categorical_cols = [col for col in full.columns if col not in continuous_cols]

# Replace NaNs with 0 in continuous columns
df_continuous = full[continuous_cols].fillna(0)

# Perform mode imputation for categorical columns
mode_imputer = SimpleImputer(strategy='most_frequent')
df_categorical = pd.DataFrame(mode_imputer.fit_transform(full[categorical_cols]), columns=categorical_cols)

# Reset the index of the DataFrames
df_continuous.reset_index(drop=True, inplace=True)
df_categorical.reset_index(drop=True, inplace=True)

# Combine the imputed dataframes
df_child = pd.concat([df_continuous, df_categorical], axis=1)

In [25]:
# Count the number of NaN values in each column
na_counts = df_child.isna().sum()

# Print the count
print("NaN value counts in df_child:")
print(na_counts)

NaN value counts in df_child:
mins_badminton                 0
year                           0
school_year                    0
age                            0
serial                         0
survey_year                    0
ethnicity                      0
district                       0
school_type                    0
urban_rural                    0
badminton_onceweek             0
Sports_easy                    0
Understand_sports_important    0
Trust_peers                    0
School_meals                   0
Gender                         0
IMD_child                      0
dtype: int64


In [26]:
df_child.describe()
#looks fine 

,mins_badminton,year,school_year,age,serial,survey_year,ethnicity,district,school_type,urban_rural,badminton_onceweek,Sports_easy,Understand_sports_important,Trust_peers,School_meals,Gender,IMD_child
count,399362.000000,399362.000000,399362.000000,399362.000000,399362.00000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000,399362.000000
mean,3.750457,2.384253,6.689810,10.839138,110203.78709,2.384253,1.714798,21.628883,1.525663,1.189485,0.057026,2.105138,1.336577,2.007046,1.077969,1.567400,6.125553
std,21.416588,1.104459,2.618822,3.274122,65334.32180,1.104459,1.420109,11.157705,0.523987,0.391894,0.231892,0.719166,0.552657,0.765327,0.268124,0.512963,2.759416
min,0.000000,1.000000,1.000000,0.000000,14.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,1.000000,5.000000,9.000000,53743.00000,1.000000,1.000000,13.000000,1.000000,1.000000,0.000000,2.000000,1.000000,2.000000,1.000000,1.000000,4.000000
50%,0.000000,2.000000,7.000000,11.000000,109866.00000,2.000000,1.000000,21.000000,2.000000,1.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,6.000000
75%,0.000000,3.000000,9.000000,13.000000,163894.75000,3.000000,1.000000,30.000000,2.000000,1.000000,0.000000,2.000000,2.000000,2.000000,1.000000,2.000000,9.000000
max,630.000000,4.000000,11.000000,16.000000,249639.00000,4.000000,6.000000,42.000000,3.000000,2.000000,1.000000,4.000000,4.000000,4.000000,2.000000,3.000000,10.000000


In [27]:
# List of continuous columns
continuous_cols = ['mins_badminton', 'year', 'school_year']

# Convert variables not in continuous_cols to object data type
for col in df_child.columns:
    if col not in continuous_cols:
        df_child[col] = df_child[col].astype('object')


In [28]:
df_child.dtypes

mins_badminton                 float64
year                             int64
school_year                    float64
age                             object
serial                          object
survey_year                     object
ethnicity                       object
district                        object
school_type                     object
urban_rural                     object
badminton_onceweek              object
Sports_easy                     object
Understand_sports_important     object
Trust_peers                     object
School_meals                    object
Gender                          object
IMD_child                       object
dtype: object

# Detecting outliers 

In [29]:
# Set the threshold for outlier detection
iqr_threshold = 1.5

# Calculate the IQR for each column
q1 = df_child.quantile(0.25)
q3 = df_child.quantile(0.75)
iqr = q3 - q1

# Define the lower and upper bounds for outliers
lower_bound = q1 - (iqr_threshold * iqr)
upper_bound = q3 + (iqr_threshold * iqr)

# Identify outliers
outliers = ((df_child< lower_bound) | (df_child > upper_bound)).any(axis=1)

# Print the rows containing outliers
print(df_child[outliers])

/tmp/ipykernel_161035/3329689108.py:5: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  q1 = df_child.quantile(0.25)
/tmp/ipykernel_161035/3329689108.py:6: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  q3 = df_child.quantile(0.75)
/tmp/ipykernel_161035/3329689108.py:14: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = ((df_child< lower_bound) | (df_child > upper_bound)).any(axis=1)


        mins_badminton  year  school_year   age    serial survey_year  \
38                 5.0     1          4.0   8.0     246.0         1.0   
56                10.0     1         11.0   0.0     279.0         1.0   
57                80.0     1         11.0  15.0     280.0         1.0   
127               15.0     1          4.0   8.0     408.0         1.0   
130              100.0     1          9.0  13.0     422.0         1.0   
...                ...   ...          ...   ...       ...         ...   
399264            40.0     4          4.0   9.0  180471.0         4.0   
399296             5.0     4          7.0  12.0  201905.0         4.0   
399342            40.0     4          7.0  12.0  143059.0         4.0   
399347            90.0     4         10.0  14.0  201646.0         4.0   
399358           310.0     4          7.0  12.0  191843.0         4.0   

       ethnicity district school_type urban_rural badminton_onceweek  \
38           1.0      1.0         3.0         1.0  

In [30]:
# Function to check the distribution of each variable
def check_distribution(df, variable, alpha=0.05):
    # Checking if the variable is normally distributed
    # H0: Data is drawn from normal distribution
    _, p_value = stats.normaltest(df[variable])
    
    if p_value < alpha:
        # If the null hypothesis can be rejected, the data is not normally distributed
        print(f"Variable {variable} is not normally distributed")
        
        # Check the distribution after log transformation
        transformed, _ = stats.boxcox(df[variable]+1)  # boxcox requires positive values
        _, p_value_transformed = stats.normaltest(transformed)
        
        if p_value_transformed > alpha:
            print(f"Variable {variable} is normally distributed after transformation\n")
        else:
            print(f"Variable {variable} is not normally distributed even after transformation\n")
    else:
        print(f"Variable {variable} is normally distributed\n")

# Iterating over each numeric column in the DataFrame
for column in df_child.select_dtypes(include=np.number).columns:
    check_distribution(df_child, column)

Variable mins_badminton is not normally distributed
Variable mins_badminton is not normally distributed even after transformation

Variable year is not normally distributed
Variable year is not normally distributed even after transformation

Variable school_year is not normally distributed
Variable school_year is not normally distributed even after transformation



In [31]:
# save the merged data to a new file
df_child.to_csv('child_df.csv', index=False)